In [1]:
%%capture
import sys
import numpy as np

%pip install pyomo >/dev/null 2>/dev/null
%pip install highspy >/dev/null 2>/dev/null

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

import pandas as pd
import csv

In [ ]:
#Importing the data
activity_table = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Activities 2")
dist_matrix = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Distances 2")

num_scenarios = 20
duration_scenarios = []

for i in range(num_scenarios):
    scenario = {}
    for j in activity_table["Activity"]:
        mu = activity_table.loc[activity_table["Activity"] == j, "Duration"].values[0]
        sigma = mu * 0.2
        duration = max(0.1, np.random.normal(mu, sigma))
        scenario[j] = duration
    duration_scenarios.append(scenario)
    

In [2]:
#Initialize the model
model = pyo.ConcreteModel("National Park Trip")

#Importing the data
activity_table = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Activities 2")
dist_matrix = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Distances 2")

# Sets
#activity_table = activity_table.set_index("Activity")
model.I = pyo.Set(initialize=activity_table['Activity'].dropna().tolist()) #set of activities
#model.P = pyo.Set(initialize=['Hiking', 'Adventures', 'Shopping', 'Driving', 'Vistas']) # category of activities
model.K = pyo.Set(initialize=[1,2,3]) #set of days


#Parameters
duration_dict = activity_table.set_index("Activity")["Duration"].to_dict() #duration of each activity
activityType_dict = activity_table.set_index("Activity")["Category"].to_dict() #type of each activity
cost_dict = activity_table.set_index("Activity")["Cost"].to_dict() #cost of each activity
funPoints_dict = activity_table.set_index("Activity")["Fun Points"].to_dict() #fun points of each activity


model.duration = pyo.Param(model.I, initialize=duration_dict)
model.activity_type = pyo.Param(model.I, initialize=activityType_dict, within=pyo.Any)
model.cost = pyo.Param(model.I, initialize=cost_dict)
model.fun_points = pyo.Param(model.I, initialize=funPoints_dict)


#Variables
model.x = pyo.Var(model.I, model.K, within=pyo.Binary) #activity selection, which activity to do on which day

#Constraints
def hiking_time(model, k):
    return sum(model.duration[i] * model.x[i, k] for i in model.I if model.activity_type[i] == 'Hiking') <= 5
model.hiking_time = pyo.Constraint(model.K, rule=hiking_time)

def adventure_budget(model, k):
    return sum(model.cost[i] * model.x[i, k] for i in model.I if model.activity_type[i] == 'Adventures') <= 600
model.adventure_budget = pyo.Constraint(model.K, rule=adventure_budget)

#set fun points per day to 10
def min_fun_per_day(model, k):
    return sum(model.fun_points[i] * model.x[i, k] for i in model.I) >= 10
model.min_fun_per_day = pyo.Constraint(model.K, rule=min_fun_per_day)


def min_activities(model):
    return sum(model.x[i, k] for i in model.I for k in model.K) >= 1
model.min_activities = pyo.Constraint(rule=min_activities)

# #Constraint to avoid selecting the same activity on different days
def same_activity(model, i):
    return sum(model.x[i, k] for k in model.K) <= 1
model.same_activity = pyo.Constraint(model.I, rule=same_activity)

# #Constraint activity time per day to 10 hours
def activity_max_time(model, k):
    return sum(model.duration[i] * model.x[i, k] for i in model.I) <= 8
model.activity_max_time = pyo.Constraint(model.K, rule=activity_max_time)

# #Constraint to avoid selecting the same activity on the same day
def activity_min_time(model,k):
    return sum(model.duration[i] * model.x[i, k] for i in model.I) >= 6
model.activity_min_time = pyo.Constraint(model.K, rule=activity_min_time)

# #Minimum number of activities per day
def min_activities_per_day(model, k):
    return sum(model.x[i, k] for i in model.I) >= 1
model.min_activities_per_day = pyo.Constraint(model.K, rule=min_activities_per_day)


#Objective Function
def total_cost(model):
    activity_cost = sum(model.x[i,k] * activity_table.at[i, 'Cost'] for i in model.I for k in model.K)
    return activity_cost

model.obj = pyo.Objective(rule=total_cost, sense=pyo.minimize)

#Solve the model
SOLVER.solve(model)
model.display()
#Extract the results
selected_activities = []
for i in model.I:
    for k in model.K:
        if pyo.value(model.x[i, k]) > 0:
            selected_activities.append(i)

#Print which activity is happening on which day 
fun_p = 0
for k in model.K:
    print(f"Day {k}:")
    for i in model.I:
        if pyo.value(model.x[i, k]) > 0:
            print(f"  - {activity_table.at[i, 'Name']} (Duration: {activity_table.at[i, 'Duration']} hours)")
            fun_p += activity_table.at[i, 'Fun Points']

    print()

print(f"Total Fun Points: {fun_p}")

Model National Park Trip

  Variables:
    x : Size=48, Index=I*K
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         (0, 1) :     0 :   0.0 :     1 : False : False : Binary
         (0, 2) :     0 :   0.0 :     1 : False : False : Binary
         (0, 3) :     0 :   0.0 :     1 : False : False : Binary
         (1, 1) :     0 :   0.0 :     1 : False : False : Binary
         (1, 2) :     0 :   1.0 :     1 : False : False : Binary
         (1, 3) :     0 :   0.0 :     1 : False : False : Binary
         (2, 1) :     0 :   0.0 :     1 : False : False : Binary
         (2, 2) :     0 :   0.0 :     1 : False : False : Binary
         (2, 3) :     0 :   0.0 :     1 : False : False : Binary
         (3, 1) :     0 :   0.0 :     1 : False : False : Binary
         (3, 2) :     0 :   0.0 :     1 : False : False : Binary
         (3, 3) :     0 :   1.0 :     1 : False : False : Binary
         (4, 1) :     0 :   0.0 :     1 : False : False : Binary
         (4, 2) :     0 